# Text Pre-processing

Add text here explaining this notebook and what it does

##### Import libraries

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import re
import string

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

## Load data

In [2]:
# Read in combined data frames csv file
posts = pd.read_csv('../data/subreddit_posts_combined.csv') 

## Look at the data

In [3]:
posts.head()

,author,created_utc,subreddit,text,timeframe
0,sub30605,1499390694,bulimia,Chest Pains? : Anyone else experience chest pa...,pre-covid
1,sub27274,1499060654,bulimia,"Dying to eat, eating to die: study on shifting...",pre-covid
2,sub6055,1499029087,bulimia,Without purging.... : What is the quickest way...,pre-covid
3,sub40365,1498978259,bulimia,Bulimia and Melancholy Feelings : I haven't pu...,pre-covid
4,sub49857,1498814187,bulimia,I'm relapsing : Fuck. I'm so upset at myself r...,pre-covid


#### Check Shape

In [4]:
posts.shape

(85401, 5)

#### Check data types

In [5]:
posts.dtypes

author         object
created_utc    object
subreddit      object
text           object
timeframe      object
dtype: object

#### Check Null Values

In [6]:
posts.isnull().sum()

author         0
created_utc    0
subreddit      1
text           1
timeframe      2
dtype: int64

In [7]:
(posts.loc[posts['text'].isnull() == True]).head(10)

,author,created_utc,subreddit,text,timeframe
27249,Pray for me.,pre-covid,NaN,NaN,NaN


In [8]:
posts.dropna(inplace=True)

In [9]:
posts.shape

(85399, 5)

#### Count number of posts per subreddit

In [10]:
posts['subreddit'].value_counts()

bulimia            9777
AnorexiaNervosa    9733
schizophrenia      9549
bipolar            9493
Anxiety            9455
mentalhealth       9450
BPD                9345
depression         9318
autism             9279
Name: subreddit, dtype: int64

## Preprocess Text

### Normalization

https://towardsdatascience.com/cleaning-text-data-with-python-b69b47b97b76

In [11]:
stop_words = stopwords.words("english")

# Add our targets to the list of stopwords 
newStopWords = ['bulimia', 'bulimic', 'anorexia', 'anorexic', 'nervosa',
                'schizophrenia', 'schizophrenic', 'bipolar', 'anxiety', 
                'mentalhealth', 'bpd', 'borderline', 'depression', 'autism',
                'autistic']

for i in newStopWords:
    stop_words.append(i)

words = set(nltk.corpus.words.words())

In [12]:
def text_preproc(x):
    x = x.lower()
    x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    x = x.encode('ascii', 'ignore').decode("utf-8")
    x = re.sub(r'https*\S+', ' ', x)
    x = re.sub(r'@\S+', ' ', x)
    x = re.sub(r'#\S+', ' ', x)
    x = re.sub(r'\'\w+', '', x)
    x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    x = re.sub(r'\w*\d+\w*', '', x)
    x = re.sub(r'\s{2,}', ' ', x)
    x = " ".join(w for w in nltk.wordpunct_tokenize(x) if w.lower() \
                 in words or not w.isalpha())
    return x

In [13]:
posts['text_clean'] = posts.text.apply(text_preproc)

In [14]:
posts.head()

,author,created_utc,subreddit,text,timeframe,text_clean
0,sub30605,1499390694,bulimia,Chest Pains? : Anyone else experience chest pa...,pre-covid,chest anyone else experience chest they purgin...
1,sub27274,1499060654,bulimia,"Dying to eat, eating to die: study on shifting...",pre-covid,dying eat eating die study shifting coping men...
2,sub6055,1499029087,bulimia,Without purging.... : What is the quickest way...,pre-covid,without purging way lose weight without exercise
3,sub40365,1498978259,bulimia,Bulimia and Melancholy Feelings : I haven't pu...,pre-covid,melancholy little month since i slowly losing ...
4,sub49857,1498814187,bulimia,I'm relapsing : Fuck. I'm so upset at myself r...,pre-covid,i relapsing i upset right now i twice week goo...


### Tokenize sentences and words

In [15]:
# tokenize sentences
posts['sent_tokens'] = [sent_tokenize(row) for row in posts['text_clean'].astype(str)] 

In [16]:
# tokenize words
posts['word_tokens'] = [word_tokenize(row) for row in posts['text_clean'].astype(str)]

### Check for empty cells

In [17]:
posts.replace('', np.nan, inplace=True) 

In [18]:
posts.isnull().sum()

author           0
created_utc      0
subreddit        0
text             0
timeframe        0
text_clean     448
sent_tokens      0
word_tokens      0
dtype: int64

In [19]:
posts.dropna(inplace=True)

### Porter Stemmer

Using Porter Stemmer

In [24]:
# Create an instance of Porter Stemmer
stemmer = PorterStemmer()

In [25]:
text = posts['word_tokens'].astype(str)

In [26]:
posts['word_tokens'] = [stemmer.stem(word) for word in text]

### More cleaning

### Check dataframe filtered by submission Lengths

In [27]:
posts.head()

,author,created_utc,subreddit,text,timeframe,text_clean,sent_tokens,word_tokens
0,sub30605,1499390694,bulimia,Chest Pains? : Anyone else experience chest pa...,pre-covid,chest anyone else experience chest they purgin...,[chest anyone else experience chest they purgi...,"['chest', 'anyone', 'else', 'experience', 'che..."
1,sub27274,1499060654,bulimia,"Dying to eat, eating to die: study on shifting...",pre-covid,dying eat eating die study shifting coping men...,[dying eat eating die study shifting coping me...,"['dying', 'eat', 'eating', 'die', 'study', 'sh..."
2,sub6055,1499029087,bulimia,Without purging.... : What is the quickest way...,pre-covid,without purging way lose weight without exercise,[without purging way lose weight without exerc...,"['without', 'purging', 'way', 'lose', 'weight'..."
3,sub40365,1498978259,bulimia,Bulimia and Melancholy Feelings : I haven't pu...,pre-covid,melancholy little month since i slowly losing ...,[melancholy little month since i slowly losing...,"['melancholy', 'little', 'month', 'since', 'i'..."
4,sub49857,1498814187,bulimia,I'm relapsing : Fuck. I'm so upset at myself r...,pre-covid,i relapsing i upset right now i twice week goo...,[i relapsing i upset right now i twice week go...,"['i', 'relapsing', 'i', 'upset', 'right', 'now..."


In [28]:
# Create a new column with submission length
posts['text_length'] = posts['sent_tokens'].map(len)

In [29]:
# Create a new column with number of words per submission
posts['stems_word_count'] = posts['word_tokens'].map(len)

In [30]:
# Show the shortest submissions
posts.sort_values(by='text_length', ascending=True)[['sent_tokens']].head()

,sent_tokens
0,[chest anyone else experience chest they purgi...
56988,[watching phone song many external outside con...
56987,[sometimes pretend teach life re center]
56986,[ever gag reflex throughout day thought medica...
56985,[new year hear everyone mine are learn web dev...


In [38]:
# Show the shortest submissions
posts.sort_values(by='stems_word_count', ascending=True)[['word_tokens']].head()

,word_tokens
59451,['d']
5699,['r']
48634,['i']
8048,['i']
58943,['s']


In [36]:
# Show the longest submissions
posts.sort_values(by='stems_word_count', ascending=False)[['word_tokens']].head(10)

,word_tokens
20780,"['reciprocal', 'suffering', 'if', 'time', 'ple..."
48207,"['rather', 'fast', 'recovery', 'i', 'turn', 'o..."
82321,"['i', 'depressed', 'long', 'remember', 'really..."
18665,"['think', 'may', 'schizophrenia', 'please', 'h..."
84879,"['really', 'long', 'sad', 'story', 'begin', 'i..."
62675,"['little', 'almost', 'nothing', 'little', 'alm..."
65625,"['experience', 'deal', 'it', 'worth', 'long', ..."
85244,"['extremely', 'long', 'post', 'felt', 'like', ..."
39545,"['right', 'feel', 'way', 'expect', 'anyone', '..."
34511,"['job', 'girl', 'taking', 'toll', 'ill', 'star..."


In [39]:
# Drop any unecessary rows before saving the dataframe to a clean csv file
posts.drop(columns=['text','text_length','stems_word_count'], inplace=True)

## Save to csv

In [40]:
posts.to_csv(r'../data/posts-preprocessed.csv', index=False)